In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#  Load datasets
bird_essays = pd.read_csv('/content/drive/MyDrive/data/BARD/BARD_essay.csv')  # columns: index, prompts, BARD
gpt_essays = pd.read_csv('/content/drive/MyDrive/data/GPT/ChatGPT_essay.csv')    # columns: index, prompts, responses
human_essays = pd.read_csv('/content/drive/MyDrive/data/Human/human_essay_1.csv') # columns: index, essays

# Prepare the texts and add source labels
bird_essays['text'] = bird_essays['BARD']
gpt_essays['text'] = gpt_essays['responses']
human_essays['text'] = human_essays['essays']

# Add source labels
bird_essays['source'] = 1  # BIRD
gpt_essays['source'] = 0    # GPT
human_essays['source'] = 2   # Human

# Combine datasets
essays_data = pd.concat([bird_essays[['text', 'source']], gpt_essays[['text', 'source']], human_essays[['text', 'source']]], ignore_index=True)

# Clean the text (example cleaning steps)
essays_data['text'] = (
    essays_data['text'].str.replace(r'http\S+|www\S+|https\S+', '', case=False)
    .str.replace(r'<.*?>', '', case=False)
    .str.replace(r'\s+', ' ', regex=True)
    .str.lower()
)

# Prepare features and labels
X = essays_data['text']
y = essays_data['source']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Calculate max sequence length and vocabulary size
max_length = max(len(seq) for seq in X_sequences)
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding

# Pad sequences
X_padded = pad_sequences(X_sequences, maxlen=max_length)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42, stratify=y)

# TextCNN model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # 3 classes for multiclass classification

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(classification_report(y_test, y_pred, target_names=['GPT', 'BIRD', 'Human']))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


72/72 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.8349 - loss: 0.3873 - val_accuracy: 0.9250 - val_loss: 0.1615
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.9352 - loss: 0.1530 - val_accuracy: 0.9389 - val_loss: 0.1479
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - accuracy: 0.9459 - loss: 0.1222 - val_accuracy: 0.9668 - val_loss: 0.1328
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.9572 - loss: 0.1052 - val_accuracy: 0.9721 - val_loss: 0.1439
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.9680 - loss: 0.0926 - val_accuracy: 0.9686 - val_loss: 0.1781
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.9728 - loss: 0.0733 - val_accuracy: 0.9651 - val_loss: 0.2756
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.9781 - loss: 0.0759 - val_accuracy: 0.9651 - val_loss: 0.1294
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9817 - loss: 0.0664 - val_accuracy: 0.9703 - val_loss: 0.